In [ ]:
import datetime as dt

from ultralytics import YOLO

from ml_carbucks import YOLO_PRETRAINED_11L, YOLO_PRETRAINED_11N, DATA_CAR_DD_YAML, RESULTS_DIR

In [ ]:



# NOTE: Honestly I have no idea what are those YOLO11l and YOLO11n and why there are being duplicated, please explain guys 
model = YOLO(YOLO_PRETRAINED_11L)

RUN_NAME = dt.datetime.now().strftime("%Y%m%d_%H%M%S") 

start_time = dt.datetime.now()
results = model.train(
    pretrained=True,         # use pretrained weights
    seed=42,                 # random seed for reproducibility
    data=DATA_CAR_DD_YAML,
    project=str(RESULTS_DIR),
    name=RUN_NAME,
    epochs=1,                # minimal epochs for speed
    imgsz=320,               # smaller image size for faster training
    batch=1,                 # small batch size to reduce memory and speed up
    workers=0,               # minimal data loading workers
    device='0',              # use GPU if available, 'cpu' for CPU
    optimizer='AdamW',       # optimizer type
    lr0=0.01,                # initial learning rate
    patience=5,              # early stopping
    verbose=False,           # less output
    # Additional useful params:
    weight_decay=0.0005,     # weight decay for optimizer
    momentum=0.937,          # momentum for SGD/AdamW    
    # box=7.5,                 # box loss gain
    # cls=0.5,                 # class loss gain
    # dfl=1.5,                 # dfl loss gain
    # hsv_h=0.015,             # image HSV-Hue augmentation
    # hsv_s=0.7,               # image HSV-Saturation augmentation
    # hsv_v=0.4,               # image HSV-Value augmentation
    # degrees=0.0,             # image rotation augmentation
    # translate=0.1,           # image translation augmentation
    # scale=0.5,               # image scale augmentation
    # shear=0.0,               # image shear augmentation
    # perspective=0.0,         # image perspective augmentation
    # flipud=0.0,              # image flip up-down
    # fliplr=0.5,              # image flip left-right
    # mosaic=1.0,              # mosaic augmentation probability
    # mixup=0.0,               # mixup augmentation probability
    # copy_paste=0.0,          # copy-paste augmentation probability
    # # ... see YOLO docs for more options
)

end_time = dt.datetime.now()

elapsed_time = end_time - start_time

print(f"Training completed in: {elapsed_time.total_seconds()}s")

In [ ]:
from pprint import pprint
pprint(results)


pprint(results.fitness)

In [ ]:
from pathlib import Path
from typing import Any, Callable, Dict, Union

from optuna import Trial
import optuna


def get_trial_params(trial: Trial) -> Dict[str,Any]:
    epochs = trial.suggest_int('epochs', 10, 75)
    batch = trial.suggest_categorical('batch', [8, 16, 32, 64])
    imgsz = trial.suggest_categorical('imgsz', [320, 640, 960])
    lr = trial.suggest_float('lr', 1e-4, 1e-1, log=True)
    momentum = trial.suggest_float('momentum', 0.5, 0.99)
    weight_decay = trial.suggest_float('weight_decay', 1e-5, 1e-2, log=True)
    patience = trial.suggest_int('patience', 25, 50)


    return {
        'epochs': epochs,
        'batch': batch,
        'imgsz': imgsz,
        'lr0': lr,
        'momentum': momentum,
        'weight_decay': weight_decay,
        "patience": patience
    }

def create_objective(
    version: str,
    data: Path,
    name: str,
    device: str, 
)-> Callable:
    
    def objective(trial: Trial) -> float:
        params = get_trial_params(trial)
        model = YOLO(version)


        try:
            results = model.train(
                pretrained=True,
                seed=42,
                data=data,
                name=name,
                device=device,
                verbose=False,
                **params,
                save=False,
            )

            trial.set_user_attr("params", params)
            trial.set_user_attr("results", results.results_dict)

            return results.fitness[0]  # Return the primary fitness metric (e.g., mAP@50-95)

        except optuna.exceptions.TrialPruned as e:
            print("Trial pruned") # NOTE: this should be replace to logger
            raise e
        except Exception as e:
            print(f"Error in objective: {e}")
            raise e
        
    return objective

def execute_study(
    name: str,
    n_trials: int = 20,
    results_dir: Path = RESULTS_DIR,
    version: Union[Path,str] = YOLO_PRETRAINED_11N,
    data: Path = DATA_CAR_DD_YAML,
    direction: str = "maximize",
    device: str = '0',
):

    sql_path = results_dir / f"{name}.db"

    study = optuna.create_study(
        direction=direction,
        study_name=name,
        load_if_exists=True,
        storage=f"sqlite:///{sql_path}",
    )


    study.optimize(
        func=create_objective(
            version=str(version),
            data=data,
            name=name,
            device=device,
        ),
        n_trials=n_trials,
        gc_after_trial=True,
    )



# NOTE: This is how to execute hyperparameter optimization, but it takes a lot of time, so I commented it out for now

# execute_study(
#     name=f"{RUN_NAME}_optuna",
#     results_dir=RESULTS_DIR,
#     version=YOLO_PRETRAINED_11N,
#     data=DATA_CAR_DD_YAML,
#     direction="maximize",
#     device='0',
# )

# NOTE: to view optuna execute in terminal: optuna dashboard sqlite:///{sql_path}

     